## "Deploy" models with wandb/GitHub
Promote a model to the model registry and document the promotion with the GitHub Deployment API.

Note: We aren't actually deploying a model, instead we are promoting a model with the tag production candidate so that your deployment systems can take it from there. The point of this exercise is to use GitHub Actions and W&B to document this process so you have more transparency and visibility.

In [1]:
from dotenv import load_dotenv
import os
import wandb
from urllib.parse import urlencode

# load the github/ wandb api keys
load_dotenv()

True

In [2]:
api = wandb.Api()

In [4]:
api.artifact('himasha/model-registry/mnist-registry:production-candidate').download("model")

wandb:   1 of 1 files downloaded.  


'model'

In [6]:
# use the registered model artifacts
# Always initialize a W&B run to start tracking
wandb.init()

model = wandb.use_artifact('himasha/model-registry/mnist-registry:v0') 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: himasha. Use `wandb login --relogin` to force relogin


In [7]:
model.download()

KeyboardInterrupt: 

## Parameters
These will we passed as environment variables in GitHub Actions

In [ ]:
owner='hamelsmu'
repo='wandb-cicd'
registry_path = 'av-team/model-registry/BDD Semantic Segmentation'
run_path = 'av-team/mlops-course-001/'
run_id = '2wpuuc70'
pr_num = '16'
environment = 'staging'

## Promote run's model to the registry

In [ ]:
api = wandb.Api()
run = api.run(f'{run_path}/{run_id}')

art = [a for a in run.logged_artifacts() if a.type == 'model']
if art:
    assert len(art) == 1, 'More then 1 artifact of type model!'
    art[0].link(registry_path, aliases=['production candidate'])

In [ ]:
versions = api.artifact_versions('model', registry_path)
latest_model = versions[0]

# define the latest model url inside the registry
query = urlencode({'selectionPath': registry_path, 'version': latest_model.version})
registry_url = f'https://wandb.ai/{latest_model.entity}/registry/model?{query}'
registry_url


## Share the link on GitHub with deployments

In [ ]:
from ghapi.core import GhApi
gapi = GhApi(owner=owner, repo=repo)

When you are making an issue coment, you need to get the branch associated with the PR with an API call

In [ ]:
branch_name = gapi.pulls.get(pr_num).head.ref

deploy = gapi.repos.create_deployment(ref=branch_name,
                                      auto_merge=False,
                                 environment=environment,
                                 description='promote model to `deployment candidate`',
                                 payload={'run_id': run_id, 'registry_url':registry_url})

You could actually deploy your model in and report the status of success or failure back. However, we aren't going to do that and let another system handle the actual deployment from the model registry.

In [ ]:
status = gapi.repos.create_deployment_status(deployment_id=deploy.id,
                                             description=f'Tagged model for run {run_id} as a `production candidate`',
                                             environment=environment,
                                             log_url=registry_url,
                                             state='success')